# Output Parsers - Formatando saídas

Como retornar dados estruturados de um modelo?

É frequentemente útil que um modelo retorne uma saída que corresponda a um esquema específico. Um caso de uso comum é a extração de dados de um texto para inseri-los em um banco de dados ou utilizá-los em algum outro sistema subsequente. Nesta aula abordaremos algumas estratégias para obter saídas estruturadas de um modelo.

## Estruturando saídas de chat - StrOutputParser

O formatador mais simples do LangChain é o StrOutputParser. Ele é utilizado para convertermos saídas do modelo no formato de conversação para formato texto. É um atividade bem comum, levando em consideração que maior parte das llms que utilizamos com LangChain são acessadas através dos ChatModels

In [1]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', 'Você é um assistente engraçado e se chama {nome_assistente}'),
        ('human', '{pergunta}')
    ]
)

chat_template.format_messages(nome_assistente='Asimo', pergunta='Qual o seu nome?')

[SystemMessage(content='Você é um assistente engraçado e se chama Asimo', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Qual o seu nome?', additional_kwargs={}, response_metadata={})]

In [2]:
prompt = chat_template.invoke({'nome_assistente': 'Asimo', 'pergunta':'Qual o seu nome?'})
prompt

ChatPromptValue(messages=[SystemMessage(content='Você é um assistente engraçado e se chama Asimo', additional_kwargs={}, response_metadata={}), HumanMessage(content='Qual o seu nome?', additional_kwargs={}, response_metadata={})])

In [3]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()
resposta = chat.invoke(prompt)
resposta

AIMessage(content='Olá! Meu nome é Asimo. Como posso ajudar você hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 31, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e5424164-a9a0-4ccd-a0d9-1afaf1c42929-0', usage_metadata={'input_tokens': 31, 'output_tokens': 19, 'total_tokens': 50, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### StrOutputParser

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(resposta)

'Olá! Meu nome é Asimo. Como posso ajudar você hoje?'

### Dando um spoiler de chains

In [5]:
chain = chat_template | chat | output_parser

chain.invoke({'nome_assistente': 'Asimo', 'pergunta':'Qual o seu nome?'})

'Meu nome é Asimo, o assistente engraçado! Como posso ajudar você hoje?'

## Estruturando saídas mais complexas - Pydantic

### Utilizando .with_structured_output()


Esta é a maneira mais fácil e confiável de obter saídas estruturadas. O método with_structured_output() é implementado para modelos que fornecem APIs nativas para estruturar saídas, como chamadas de ferramentas/funções ou modo JSON, e aproveita essas capacidades internamente.

Este método recebe um esquema como entrada, que especifica os nomes, tipos e descrições dos atributos desejados na saída. Ele retorna um objeto similar a um Runnable, exceto que, em vez de gerar strings ou mensagens, produz objetos correspondentes ao esquema fornecido. O esquema pode ser especificado como uma classe TypedDict, um JSON Schema ou uma classe Pydantic.

In [6]:
from typing import Optional
from pydantic import BaseModel, Field

class Piada(BaseModel):
    """Piada para contar ao usuário"""
    introducao: str = Field(description='A introdução da piada')
    punchline: str = Field(description='A conclusão da piada')
    avaliacao: Optional[int] = Field(description='O quão engraçada é a piada de 1 a 10')

llm_estruturada = chat.with_structured_output(Piada)
resposta = llm_estruturada.invoke('Conte uma piada sobre gatinhos')
resposta

Piada(introducao='Por que os gatinhos são tão bons em matemática?', punchline='Porque eles têm múltiplos talentos!', avaliacao=8)

In [7]:
type(resposta.punchline)

str

### Um exemplo mais prático

Digamos que temos a seguinte review de um produto:

> "Este soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. Acho que minha esposa gostou tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usado em todas as manhãs alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras."

E eu quero que o modelo de linguagem processe esta review para estruturá-la no seguinte formato:

```json
{
  "presente": true,
  "dias_entrega": 2,
  "percepcao_de_valor": ["um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado"]
}

```



In [8]:
review_cliente = """Este soprador de folhas é bastante incrível. Ele tem 
quatro configurações: sopro de vela, brisa suave, cidade ventosa 
e tornado. Chegou em dois dias, bem a tempo para o presente de 
aniversário da minha esposa. Acho que minha esposa gostou tanto 
que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho 
usado em todas as manhãs alternadas para limpar as folhas do 
nosso gramado. É um pouco mais caro do que os outros sopradores 
de folhas disponíveis no mercado, mas acho que vale a pena pelas 
características extras."""


In [9]:
from pydantic import BaseModel, Field

class AvaliacaoReview(BaseModel):
    """Avalia review do cliente"""
    presente: bool = Field(description='Verdadeiro se foi para presente e False se não foi')
    dias_entrega: int = Field(description='Quantos dias para entrega do produto')
    percepcao_valor: list[str] = Field(description='Extraia qualquer frase sobre o valor ou \
    preço do produto. Retorne uma lista.')

llm_estruturada = chat.with_structured_output(AvaliacaoReview)
resposta = llm_estruturada.invoke(review_cliente)
resposta

AvaliacaoReview(presente=True, dias_entrega=2, percepcao_valor=['um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado', 'vale a pena pelas características extras'])

In [10]:
resposta.presente

True

In [11]:
resposta.dias_entrega

2

In [12]:
resposta.percepcao_valor

['um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado',
 'vale a pena pelas características extras']